In [14]:

import overturemaps
bbox = -74.02169, 40.696423, -73.891338, 40.831263
table = overturemaps.record_batch_reader("building", bbox, release="2025-11-19.0").read_all()
heights = table["height"].to_numpy()
heights = np.nan_to_num(heights, nan=1)


In [15]:
import pandas as pd
import pyarrow as pa
import geopandas as gpd
from shapely.geometry import Polygon, LineString
from lonboard import Map, PolygonLayer
from lonboard.experimental import TripsLayer
# from lonboard import Map, SolidPolygonLayer, PathLayer, TripsLayer
import numpy as np
# from lonboard.basemap import CartoStyle, MaplibreBasemap
from datetime import timedelta
# --- 1. Load Data ---
TRIPS_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/trips/trips-v7.json"
BUILDINGS_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/trips/buildings.json"

df_trips = pd.read_json(TRIPS_URL)
df_buildings = pd.read_json(BUILDINGS_URL)

# --- 2. Prepare Data ---

# A. Buildings
df_buildings['geometry'] = df_buildings['polygon'].apply(lambda x: Polygon(x))
gdf_buildings = gpd.GeoDataFrame(df_buildings, geometry='geometry')

# B. Trips
start_time = pd.Timestamp("2024-01-01 00:00:00")

# Create list of datetimes (This creates a 'object' column in Pandas)
df_trips['timestamps'] = df_trips['timestamps'].apply(
    lambda x: [start_time + timedelta(seconds=t) for t in x]
)

df_trips['geometry'] = df_trips['path'].apply(lambda x: LineString(x))

# Colors
def get_color_array(vendor):
    return [253, 128, 93] if vendor == 0 else [23, 184, 190]
df_trips['color_rgb'] = df_trips['vendor'].apply(get_color_array)

gdf_trips = gpd.GeoDataFrame(df_trips, geometry='geometry')

# --- 3. THE CRITICAL FIX: Convert to Arrow Array ---
# Lonboard cannot infer "List of Timestamps" from a generic Pandas object column.
# We must force it into a PyArrow Array.

timestamps_array = pa.array(gdf_trips['timestamps'])
# --- 3. Create Lonboard Layers ---

# Layer 1: 3D Buildings
# FIX: Pass the numpy array (.values) instead of the string "height"
# building_layer = PolygonLayer.from_geopandas(
#     gdf_buildings,
#     get_elevation=gdf_buildings['height'].values, 
#     get_fill_color=[74, 80, 87, 255], 
#     opacity=1.0
# )

# building_layer = PolygonLayer.from_geopandas(
#     gdf_buildings,
#     extruded=True,      # Enable 3D Extrusion
#     get_elevation=gdf_buildings['height'].values, # Pass the numpy array of heights
#     get_fill_color=[74, 80, 87, 255], 
#     opacity=0.5,
#     wireframe=True      # Optional: adds nice outlines to the 3D shapes
# )

building_layer = PolygonLayer(
    # Select only a few attribute columns from the table
    table=table.select(["id", "height", "geometry", "names"]),
    extruded=True,
    get_elevation=heights,
    get_fill_color=[74, 80, 87, 255], 
    opacity=0.5,
    wireframe=True      # Optional: adds nice outlines to the 3D shapes
)

# Layer 2: Trips Paths
trips_layer = TripsLayer.from_geopandas(
    gdf_trips,
    get_timestamps=timestamps_array, # <--- Pass the Arrow Array here
    get_color=np.stack(gdf_trips['color_rgb'].values).astype(np.uint8),
    width_min_pixels=2,
    # fade_trail = True,
    trail_length=100000,
    opacity=0.8
)

# --- 4. Render Map ---
# 1. Define the static path layer (Background)
linestring_layer = PathLayer(
    table=trips_layer.table,
    get_color=trips_layer.get_color,
    width_min_pixels=0.5,
    opacity=0.001,  # Keep this low to act as a faint trace
)

# 2. Define the Basemap
# bmap = MaplibreBasemap(mode='reverse-controlled', style=CartoStyle.DarkMatter)

# 3. Create the Map with [Background, Foreground] order
# putting linestring_layer first ensures it is drawn BELOW the trips_layer
view_state = {
    "longitude": -73.98416810282863,
    "latitude": 40.72651721370669,
    "pitch": 45,
    "zoom": 12,
    "bearing": 13,
}
carto_dark_style = "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json"
m = Map(layers=[building_layer, trips_layer], basemap_style=carto_dark_style, view_state=view_state, height=600)
m.set_view_state(pitch=45)
# 4. Display the map
display(m)

# 5. Initialize the Animation Widget
# This will display the slider/play control
trips_layer.animate(step=timedelta(seconds=1), fps=50)

/usr/local/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:40: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(
/usr/local/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:40: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(
/usr/local/lib/python3.12/site-packages/lonboard/experimental/traits.py:151: UserWarning: Reducing precision of input timestamp data to 'ms' to fit into available GPU precision.
  warnings.warn(
